In [1]:
import cv2 as cv
import numpy as np
import skimage.io as io
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage
import math
from skimage.color import rgb2gray,rgb2hsv,gray2rgb,hsv2rgb
from skimage.feature import canny
import glob
from PIL import Image
from sklearn.cluster import KMeans
from random import seed
from random import randint
import time

In [2]:
#input : frame ,initial background with no human in it
#output: binary image 
def segmentation (frame,background):
    
    background = cv.cvtColor(background, cv.COLOR_BGR2GRAY)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    diff = cv.absdiff(frame, background)
    #gray = cv.cvtColor(diff, cv.COLOR_BGR2GRAY)
    gray = diff
    x,y = gray.shape
    gray[gray > 100] = 255
    gray[gray <= 100] = 0
    return gray

In [3]:
#input binary image
#output array with border points
def border_extraction(img):
    erosion = cv.erode(img, None, iterations=2)
    Human = cv.dilate(img, None, iterations=10)
    HumanEroded = cv.erode(Human, None, iterations=2)
    window = cv.namedWindow("DIFF", cv.WINDOW_NORMAL| cv.WINDOW_KEEPRATIO )
    cv.imshow("DIFF", HumanEroded)
    #HumanEroded = Human
    #io.imshow(HumanEroded,cmap="gray")
    #io.show()
    #border = cv.absdiff(Human, HumanEroded) 
    #return np.where(border==[255])
    #border = canny(HumanEroded,low_threshold=50,high_threshold=200,sigma=0)
    check = False
    white = np.where(HumanEroded == [255])
    black=np.copy(img*0)
    if(len(white[0]) > 200):
        contours,_ = cv.findContours(HumanEroded, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)


        HumanEroded = cv.cvtColor(HumanEroded, cv.COLOR_GRAY2RGB)
        x,y,z = HumanEroded.shape
        edage = np.zeros((x,y,z))
        checkhuman = np.zeros((x,y,z))
        cnts = sorted(contours, key = cv.contourArea, reverse = True)[:5]

        HumanEroded[0:x][0:y][0:z] = 0
        cv.drawContours(edage,[cnts[0]],-1,(255,0,0),3)
        window = cv.namedWindow("edges", cv.WINDOW_NORMAL| cv.WINDOW_KEEPRATIO )
        cv.imshow("edges", edage)
        indices = np.where(edage == (255,0,0))
        cv.fillPoly(checkhuman, pts =[cnts[0]], color=(255,255,255))
        #window1 = cv.namedWindow("Human", cv.WINDOW_NORMAL| cv.WINDOW_KEEPRATIO )
        #cv.imshow("Human", checkhuman)
        Humanlimits = np.where(checkhuman == (255,255,255))
        if(len(cnts) > 20 or (np.any(Humanlimits[0] == 1) or (np.any(Humanlimits[1] == 1)) or np.any(Humanlimits[0] == 2) or (np.any(Humanlimits[1] == 2)) or np.any(Humanlimits[0] == x) or (np.any(Humanlimits[1] == y)) or np.any(Humanlimits[0] == x-1) or (np.any(Humanlimits[1] == y-1))   )== False):
            check = True
        i = 0
        #return np.where(border== True)
        #borders with find contours
        pic_bin = img.astype('uint8')
        contours = cnts[i]
        area = cv.contourArea(contours)
        #while(area > ((x*y*3)/4) and i < len(cnts)-1):
            #i = i+1
            #contours = cnts[i]
            #area = cv.contourArea(contours)
        #print(contours)
       # contours, hierarchy = cv.findContours(pic_bin,mode=cv.RETR_EXTERNAL,method=cv.CHAIN_APPROX_SIMPLE)

        cv.drawContours(black, contours, -1, (255,255,255), 10)
        black = rgb2gray(black)
        window = cv.namedWindow("Black", cv.WINDOW_NORMAL| cv.WINDOW_KEEPRATIO )
        cv.imshow("Black", black)
    
    
    return black,check

In [4]:
def centroid_2(img):
    try:
        #get 255 points
        border = (np.where(img==[255]))
        #print(border)
        max_x_img = np.shape(img)[0]

        min_y = np.min(border[0])
        max_y = np.max(border[0])

        pil_img = Image.fromarray(img)
        cropped_img = pil_img.crop((0,min_y,max_x_img,max_y))
        cropped_img = np.array(cropped_img)


        third = np.array_split(cropped_img,3)[2]
        first = np.array_split(cropped_img,3)[0]

        x_values = np.where(third==[255])[1]

        max_x = np.max(x_values)
        min_x = np.min(x_values)
        xc = min_x + abs(max_x-min_x)/2

        img = gray2rgb(img)
        cv.circle(img,(int(xc),max_y),10,(0,0,255),2)


        first_points = np.where(first==[255])
        equiv_y = first_points[0][np.where(first_points[1] == int (xc))]

        #print(x_values)
        #print(equiv_y)

        y_head = np.max(equiv_y)+min_y 
        yc = y_head+abs(max_y-y_head)/2

        cv.circle(img,(int(xc),int(yc)),10,(0,0,255),2)

        return int(xc),int(yc),y_head,max_y,True
    except:
        return -1,-1,-1,-1,False
        pass

In [5]:
def peak(img,xc,yc,tx,ty,right):
    y,x = np.where(np.array(img) == 255)
    x = x + tx
    y = y + ty
    signal = np.sqrt(pow((x-xc),2)+pow((y-yc),2))
    
   
    if(len(signal) >0):
        signal= scipy.ndimage.gaussian_filter1d(signal, 1)
        peaks = scipy.signal.find_peaks(signal)[0]
        x = x[peaks]
        y = y[peaks]
        if(len(peaks) > 0):
            if(right == True):
                maxInd = np.argmax(x)
                
                return x[maxInd],y[maxInd]
            else:
                maxInd = np.argmin(x)
               
                return x[maxInd],y[maxInd]
        else:
            return -1,-1  
    else:
        return -1,-1
    

In [6]:
#input border points
#output centroid, star points, distances from centroid to star points

def star(img,xc,yc,yhead):
    try:
         #extraction white points coordinates and calculation xc , yc
        pts = np.where(img == 255)

        x = pts[1]
        y = pts[0]
        length = len(x)

        #xc = np.sum(x)/length
        #yc = np.sum(y)/length

        #points to decide where to crop the img
        #print(img.shape)
        rows,cols = img.shape
        ind = np.where(y == yc)

        x1 = np.min(x[ind])
        x2 = np.max(x[ind])

        im = Image.fromarray(img)
        lwr = im.crop((0,yc,cols,rows))
        left = im.crop((0,0,x1,int(yc)))
        right = im.crop((x2,0,cols,int(yc)))

        #io.imshow(np.array(left),cmap="gray")
        #io.show()
        #io.imshow(np.array(lwr),cmap="gray")
        #io.show()
        #io.imshow(np.array(right),cmap="gray")
        #io.show()


        lx,ly = peak(left,xc,yc,0,0,False)
        if(lx == -1 and ly == -1):
            lx = x1
            ly = yc


        rx,ry = peak(right,xc,yc,x2,0,True)
        if(rx == -1 and ry == -1):
            rx = x2
            ry = yc

        #wx,wy = peak(lwr,xc,yc,0,yc,)

        #im = gray2rgb(img)
        #cv.circle(im,(lx,ly),7,(255,0,0),2)
        #cv.circle(im,(rx,ry),7,(0,255,0),2)
        #cv.circle(im,(xc,yhead),7,(0,0,255),2)
        #cv.circle(im,(wx,wy),7,(255,0,255),2)

        #io.imshow(im)
        #io.show()
        return (lx,ly),(xc,yhead),(rx,ry),True
    except:
        return (-1,-1),(-1,-1),(-1,-1),False
        pass
        
    

In [7]:
def getAngles(stars,xc,yc):
    angle=np.zeros(2)
    angle[0]=math.degrees( math.atan2(stars[2][1]-yc, stars[2][0] -xc) - math.atan2(stars[0][1]-yc,stars[0][0]-xc))
    if(angle[0]<0):
        angle[0]=angle[0]+360
    angle[1] = math.degrees(   math.atan2(stars[1][1]-yc,stars[1][0]-xc)-math.atan2(stars[2][1]-yc, stars[2][0] -xc))
    if(angle[1]<0):
        angle[1]=angle[1]+360
    return angle

In [8]:
def getAngle(arm,head,xc,yc):
    angle=np.zeros(1)
    angle[0]=math.degrees( math.atan2(head[1]-yc, head[0] -xc) - math.atan2(arm[1]-yc,arm[0]-xc))
    if(angle[0]<0):
        angle[0]=angle[0]+360
    return angle

In [9]:
def distinguish_points(star_points,centroid):
    #problem could sholder be detected instea of han in 9 or 10
    #print('distinguish')
    #get head and both arms
    #get right hand
    max_0 = np.max(star_points,axis=0)
    right_idx = np.where((star_points[:,0]==max_0[0]))[0]
    right_point = star_points[right_idx][0]
    star_points = np.delete(star_points,right_idx[0],axis=0)

    #get left hand
    min_0 = np.min(star_points,axis=0)
    left_idx = np.where((star_points[:,0]==min_0[0]))[0]
    left_point = star_points[left_idx][0]
    star_points = np.delete(star_points,left_idx[0],axis=0)
    
    #get head
  
    flag = True
    #not sure if it is accurate
    while(flag):
        min_0 = np.min(star_points,axis=0)
        head_idx = np.where((star_points[:,1]==min_0[1]))[0]
        head_point = star_points[head_idx][0]
        star_points = np.delete(star_points,head_idx[0],axis=0)
        
        if(abs(head_point[0]-centroid[0])<20):
            flag = False
    
    return right_point,left_point,head_point

In [10]:
def detect_right_hand (right_cluster,point):
    #there're  6 clusters :
    #0 -> arm diagonal up
    #1 -> arm bent above head
    #2 -> arm diagonal down
    #3 -> 
    #4 -> arm bent at waist
    #5 -> 
    
    point=point.reshape(1,-1)
    predicted =right_cluster.predict(point)
    #print(predicted)
    return predicted
    
    #return which label


In [11]:
def detect_left_hand (left_cluster,point):
    #there're  6 clusters :
    #0 -> arm bent at waist
    #1 -> 
    #2 -> 
    #3 -> T-shape 
    #4 -> 
 
   
    point=point.reshape(1,-1)
    predicted = left_cluster.predict(point)
    #print(predicted)
    return predicted
    #return which label


In [12]:
'''
def detect_pose(right_hand_pose,left_hand_pose,pose,R,L):
    flag = False
    if(R<35 and L<35):
        if(pose == 1):
            if(right_hand_pose==0 and left_hand_pose==4):
                flag = True
        if(pose == 2):
            if(right_hand_pose==3 and left_hand_pose==1):
                flag = True
        if(pose == 3):
            if(right_hand_pose==1 and left_hand_pose==1):
                flag = True
        if(pose == 4):
            if(right_hand_pose==0 and left_hand_pose==2):
                flag = True
        if(pose == 5):
            if(right_hand_pose==1 and left_hand_pose==2):
                flag = True
        if(pose == 6):
            if(right_hand_pose==0 and left_hand_pose==1):
                flag = True
        if(pose == 7):
            if(right_hand_pose==4 and left_hand_pose==0):
                flag = True
        if(pose == 8):
            if(right_hand_pose==2 and left_hand_pose==1):
                flag = True
        if(pose == 9):
            if(right_hand_pose==3 and left_hand_pose==0):
                flag = True
        if(pose == 10):
            if(right_hand_pose==4 and left_hand_pose==4):
                flag = True
        if(pose == 11):
            if(right_hand_pose==2 and left_hand_pose==3):
                flag = True
            
    return flag
'''

'\ndef detect_pose(right_hand_pose,left_hand_pose,pose,R,L):\n    flag = False\n    if(R<35 and L<35):\n        if(pose == 1):\n            if(right_hand_pose==0 and left_hand_pose==4):\n                flag = True\n        if(pose == 2):\n            if(right_hand_pose==3 and left_hand_pose==1):\n                flag = True\n        if(pose == 3):\n            if(right_hand_pose==1 and left_hand_pose==1):\n                flag = True\n        if(pose == 4):\n            if(right_hand_pose==0 and left_hand_pose==2):\n                flag = True\n        if(pose == 5):\n            if(right_hand_pose==1 and left_hand_pose==2):\n                flag = True\n        if(pose == 6):\n            if(right_hand_pose==0 and left_hand_pose==1):\n                flag = True\n        if(pose == 7):\n            if(right_hand_pose==4 and left_hand_pose==0):\n                flag = True\n        if(pose == 8):\n            if(right_hand_pose==2 and left_hand_pose==1):\n                flag = True\n

In [13]:
def detect_pose(right_hand_pose,left_hand_pose,R,L):
    flag = False
    p=-1
    if(R<35 and L<35):
        if(right_hand_pose==0 and left_hand_pose==4):
            flag = True
            p=1
            
        if(right_hand_pose==3 and left_hand_pose==1):
            flag = True
            p=2
        if(right_hand_pose==1 and left_hand_pose==1):
            flag = True
            p=3
        if(right_hand_pose==0 and left_hand_pose==2):
            flag = True
            p=4
        if(right_hand_pose==1 and left_hand_pose==2):
            flag = True
            p=5
        
        if(right_hand_pose==0 and left_hand_pose==1):
            flag = True
            p=6
            
        if(right_hand_pose==4 and left_hand_pose==0):
            flag = True
            p=7
            
    
        if(right_hand_pose==2 and left_hand_pose==1):
            flag = True
            p=8
            
    
        if(right_hand_pose==3 and left_hand_pose==0):
            flag = True
            p=9
            
        
        if(right_hand_pose==4 and left_hand_pose==4):
            flag = True
            p=10
        
        if(right_hand_pose==2 and left_hand_pose==3):
            flag = True
            p=11
    return flag,p

In [14]:
def resiz(left,right,xc,yc,height):
    
    distLeft=distance(left,xc,yc)
    distRight=distance(right,xc,yc)
    factor= 124/height
    leftNew = distLeft*factor
    rightNew= distRight*factor
    diff1=leftNew-distLeft
    diff2=rightNew-distRight
    v1=np.zeros(2)
    v2=np.zeros(2)
    v1[0]=left[0]-xc
    v1[1]=left[1]-yc
    v2[0]=right[0]-xc
    v2[1]=right[1]-yc
    vNorm1=v1/distLeft
    vNorm2=v2/distRight
    newLeft= left+(diff1*vNorm1)
    newRight= right+(diff2*vNorm2)
    return newLeft,newRight
    

In [15]:
def distance(left,xc,yc):
    
    dist=np.sqrt(((left[0]-xc)**2)+((left[1]-yc)**2))
    
    return dist

In [16]:
def cluster_sequence(fram,back):

    

    #border extraction step from bin_img
    #border = border_extraction(bin_img)
    borderImg = cluster_seg(fram,back)
    xc,yc,y_head,leg_height,flag2 = centroid_2(borderImg)
   
    left_arm,head,right_arm,flag2= star(borderImg,xc,yc,y_head)
    
    #get angles
    #distinguish points 
    #right_arm,left_arm,head = distinguish_points(star_points,centroid)
    
    #show points
    col_img =gray2rgb(borderImg)
    cv.circle(col_img,((int)(right_arm[0]),(int)(right_arm[1])),10,(255,0,255),2)
    cv.circle(col_img,((int)(left_arm[0]),(int)(left_arm[1])),10,(255,0,255),2)
    cv.circle(col_img,((int)(head[0]),(int)(head[1])),10,(255,0,255),2)
    #export jpg
    
    ############
    
    height = abs(leg_height-yc)
    newL,newR= resiz(left_arm,right_arm,xc,yc,height)
    left_arm=newL
    right_arm=newR
    ##################
    right_angle = getAngle(right_arm,head,xc,yc)
    left_angle = getAngle(left_arm,head,xc,yc)
   
    #################
    right_arm[0]=right_arm[0]-xc
    right_arm[1]=right_arm[1]-yc
    left_arm[0]=left_arm[0]-xc
    left_arm[1]=left_arm[1]-yc
    #############################################
    return np.concatenate((right_arm,right_angle)),np.concatenate((left_arm,left_angle)),col_img
    #return col_img
    
    #right_hand_pose = detect_right_hand(r_cluster,right_arm,right_angle)
    #left_hand_pose = detect_left_hand(l_cluster,left_arm,left_angle)
    #detect_pose(right_hand_pose,left_hand_pose,pose)

In [17]:
def cluster_seg(frame,background):
    diff = cv.absdiff(frame, background)
    gray = cv.cvtColor(diff, cv.COLOR_BGR2GRAY)
    x,y = gray.shape
    gray[gray > 35] = 255
    gray[gray <= 35] = 0
    img = gray
    Human = cv.dilate(img, None, iterations=5)
    HumanEroded = cv.erode(Human, None, iterations=2)

    pic_bin = img.astype('uint8')
    contours, hierarchy = cv.findContours(pic_bin,mode=cv.RETR_EXTERNAL,method=cv.CHAIN_APPROX_SIMPLE)

    black=np.copy(img*0)
    cv.drawContours(black, contours, -1, (255,255,255), 2)
    black = rgb2gray(black)
    return black 


In [18]:
def cluster():
    image_list = []
    right_list = []
    left_list = []
    name_list = []

    background = io.imread ("T.1_back.jpg")
    
    for filename in glob.glob('pictures/*.jpg'): #assuming gif
        im=io.imread(filename)
        image_list.append(im)

        right_point,left_point,out_img = cluster_sequence(im,background)
        right_list.append(right_point)
        
        name_list.append(filename)
        left_list.append(left_point)
        io.imsave('star_'+filename,out_img)

    background = io.imread ("blck.jpg")
    for filename in glob.glob('human/*.jpg'): #assuming gif
        im=io.imread(filename)
        image_list.append(im)
        
        right_point,left_point,out_img = cluster_sequence(im,background)
        right_list.append(right_point)
      
        name_list.append(filename)
        left_list.append(left_point)
        io.imsave('star_'+filename,out_img)
    background = io.imread ("back.jpg")
    '''
    for filename in glob.glob('salma/*.jpg'): #assuming gif
        im=io.imread(filename)
        image_list.append(im)
        io.imshow(im)
        io.show()
       
        
        
        right_point,left_point,out_img = cluster_sequence(im,background)
        right_list.append(right_point)
        
        name_list.append(filename)
        left_list.append(left_point)
        io.imsave('star_'+filename,out_img)
        '''

    cluster_n = 5
    
    right_cluster=KMeans(cluster_n,random_state=0).fit(right_list)
    
    left_cluster=KMeans(cluster_n,random_state=0).fit(left_list)
    
    
    #for i in range (0,len(image_list)):
    #    io.imshow(image_list[i])
     #   io.show()
      #  print(right_cluster.labels_[i])
       # print(left_cluster.labels_[i])
    
   # predictError(name_list,right_list,right_cluster,left_list,left_cluster)
    
    return right_cluster,left_cluster

In [19]:
def predictError(names,right,right_cluster,left,left_cluster):
    varL=[]
    varR=[]
    
    for i in range(len(names)):
       
        lab= right_cluster.labels_[i]
        cent=right_cluster.cluster_centers_[lab]
        varR.append(np.sqrt(((right[i][0]-cent[0])**2)+((right[i][1]-cent[1])**2)+((right[i][2]-cent[2])**2)))
        #io.imshow(names[i])
        #io.show()
        #print(names[i])
        #print(lab)
        #print(var)
        labL= left_cluster.labels_[i]
        centL=left_cluster.cluster_centers_[labL]
        varL.append(np.sqrt(((left[i][0]-centL[0])**2)+((left[i][1]-centL[1])**2)+((left[i][2]-centL[2])**2)))
    
   # print(varR,varL)     
    

In [20]:
def getSigma(right,right_cluster,left,left_cluster,right_l,left_l):
    
    centR= right_cluster.cluster_centers_[right_l]
    centR=centR[0]
    #print(centR)
    varR =np.sqrt(((right[0]-centR[0])**2)+((right[1]-centR[1])**2)+((right[2]-centR[2])**2))
    #print(var)
    
    centL= left_cluster.cluster_centers_[left_l]
    centL=centL[0]
    #print(left_l)
    #print(centR,centL)
    varL =np.sqrt(((left[0]-centL[0])**2)+((left[1]-centL[1])**2)+((left[2]-centL[2])**2))
    #print(var)
    #############
    return varR,varL

In [21]:
#collective function
def functions_sequence (fram,back,right_cluster,left_cluster,pose): 
    flag = False
    #get the background and keep it
    #call get a frame and call sequence of functions
    
    #segmentation step
    #im1 = io.imread ("pictures/5 (2).jpg")
    #im2 = io.imread ("pictures/T.1_back.jpg")

    bin_img = segmentation(fram,back)

    #border extraction step from bin_img
    #border = border_extraction(bin_img)
    borderImg,check = border_extraction(bin_img)
    
    if(check == True):
        xc,yc,y_head,leg_height,flag2 = centroid_2(borderImg)
    
    #star skeletonization step

        if(flag2):
            left_arm,head,right_arm,flag2= star(borderImg,xc,yc,y_head)
            if(flag2):

            #show points
                col_img =gray2rgb(borderImg)
                cv.circle(col_img,((int)(right_arm[0]),(int)(right_arm[1])),10,(255,0,255),2)
                cv.circle(col_img,((int)(left_arm[0]),(int)(left_arm[1])),10,(255,0,255),2)
                cv.circle(col_img,((int)(head[0]),(int)(head[1])),10,(255,0,255),2)
                #export jpg
                window = cv.namedWindow("Star", cv.WINDOW_NORMAL| cv.WINDOW_KEEPRATIO )
                cv.imshow("Star", col_img)

                ############

                height = abs(leg_height-yc)
                newL,newR= resiz(left_arm,right_arm,xc,yc,height)
                left_arm=newL
                right_arm=newR
                ##################
                right_angle = getAngle(right_arm,head,xc,yc)

                left_angle = getAngle(left_arm,head,xc,yc)

                #################
                right_arm[0]=right_arm[0]-xc
                right_arm[1]=right_arm[1]-yc
                left_arm[0]=left_arm[0]-xc
                left_arm[1]=left_arm[1]-yc
                #############################################
                right = np.concatenate((right_arm,right_angle))
                left = np.concatenate((left_arm,left_angle))

                right_hand_pose = detect_right_hand(right_cluster,right)
                left_hand_pose = detect_left_hand(left_cluster,left)

                
                R,L = getSigma(right,right_cluster,left,left_cluster,right_hand_pose,left_hand_pose)

                flag = detect_pose(right_hand_pose,left_hand_pose,R,L)
            else:
                flag = False,-1
        else:
            flag = False,-1
    else:
        flag = False,-1
    return flag
    #return col_img
    
    #right_hand_pose = detect_right_hand(r_cluster,right_arm,right_angle)
    #left_hand_pose = detect_left_hand(l_cluster,left_arm,left_angle)
    #detect_pose(right_hand_pose,left_hand_pose,pose)

In [22]:
#main
#get background
#im2 = io.imread ("T.1_back.jpg")
#right_cluster,left_cluster = cluster()
#right_cluster,left_cluster = cluster()
#randomly generate target pose
#pose = 1
############video loop##############
#im1 = io.imread ("pictures/3 (12).jpg")
#right,left,out=functions_sequence(im1,im2)
#right_hand_pose = detect_right_hand(right_cluster,right)
#left_hand_pose = detect_left_hand(left_cluster,left)
#detect_pose(right_hand_pose,left_hand_pose,pose)

#save image with star if required
#io.imsave('out_pictures/out.jpg',out)

############video loop##############

In [ ]:
#right_cluster,left_cluster = cluster()

pose = 11
right_cluster,left_cluster = cluster()
cap = cv.VideoCapture(0)
cap.set(cv.CAP_PROP_AUTOFOCUS, 0)

ret, Background = cap.read()
flag = False

while(True):
    ret, frame = cap.read()
    f=False
    f,p=functions_sequence (frame,Background,right_cluster,left_cluster,pose)
    if(True):
        print(p)
        cv.putText(frame,str(p),(0,120),cv.FONT_HERSHEY_SIMPLEX,2,(255,0,0),3,cv.LINE_AA)
        cv.imshow('video',frame )
        poseim3 = io.imread("poses/"+str(p)+".jpg")
        cv.imshow('pose',poseim3)
        
    check = False
    #export jpg00
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()


/home/mariamheiba/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: star_pictures/7 (3).jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mariamheiba/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: star_pictures/7 (5).jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mariamheiba/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: star_pictures/3 (10).jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mariamheiba/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: star_pictures/7 (4).jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mariamheiba/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: star_pictures/3 (12).jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mariamheiba/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py

/home/mariamheiba/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: star_pictures/4.6.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mariamheiba/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: star_pictures/1 (2).jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mariamheiba/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: star_pictures/2 (2).jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mariamheiba/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: star_pictures/2 (3).jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mariamheiba/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: star_pictures/1 (1).jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mariamheiba/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
9
-1
-1
2
2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


In [ ]:
'''
right_cluster,left_cluster = cluster()

#pose = 1
# seed random number generator
seed(1)
# generate some integers
pose = randint(0, 10)

cap = cv.VideoCapture(0)
cap.set(cv.CAP_PROP_AUTOFOCUS, 0)

ret, Background = cap.read()
counter = 0
framecounter = 0
mc = 0
flag = False
newpose = True
correct = io.imread("correct.png")
wrong = io.imread("wrong.PNG")
while(True):
    if(counter > 10):
        if(mc >= 3 ):
            cv.imshow('result', correct)
            #sleep(3)
            #cv.destroyWindow("result")
            newpose = True
        else:
            mc = 0
            counter = 0
            newpose = False
            
    if(framecounter % 100 == 0 or newpose):
        if(framecounter != 0 and newpose == False):
            cv.imshow('result', wrong)
            #sleep(3)
            #cv.destroyWindow("result")
        pose1 =  3  #randint(1, 11)
        pose = pose1
        #pose2 = randint(1, 11)
        #pose3 = randint(1, 11)
        poseim1 = io.imread("poses/"+str(pose1)+".jpg")
        #poseim2 = io.imread("poses/"+str(pose2)+".jpg")
        #poseim3 = io.imread("poses/"+str(pose2)+".jpg")
        #numpy_vertical_concat = np.concatenate((poseim1, poseim2,poseim3), axis=1)
        numpy_vertical_concat = cv.resize(poseim1, (200, 100))
        cv.imshow('Poses', poseim1)
        #time.sleep(10)

        #hide = randint(1, 3)
        #if(hide == 1):
            #numpy_vertical_concat = np.concatenate((poseim2,poseim3), axis=1)
            #pose = pose1
        #if(hide == 2):
            #numpy_vertical_concat = np.concatenate((poseim1,poseim3), axis=1)
            #pose = pose2
        #if(hide == 3):
            #numpy_vertical_concat = np.concatenate((poseim1, poseim2), axis=1)
            #pose = pose3
        
        #numpy_vertical_concat = cv.resize(numpy_vertical_concat, (960, 540))
        #cv.imshow('Poses', numpy_vertical_concat)

    ret, frame = cap.read()
    done = False
    done = functions_sequence (frame,Background,right_cluster,left_cluster,pose)
    
    window = cv.namedWindow("frame", cv.WINDOW_NORMAL| cv.WINDOW_KEEPRATIO )
    cv.imshow("frame", frame)
    print(done)
    if(done == True):
        io.imshow(frame)
        mc = mc + 1
    check = False
    counter = counter +1
    framecounter = framecounter + 1
    #cv.imshow('result', wrong)
    #export jpg
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()
'''

In [ ]:
''''
#right_cluster,left_cluster = cluster()

pose = 1
right_cluster,left_cluster = cluster()

Background = io.imread("back.jpg")
flag = False
frame = io.imread("salma/2019-12-16-172341.jpg")

f=functions_sequence (frame,Background,right_cluster,left_cluster,pose)
if(f==True):
    print(f)
io.imshow(frame)
io.show()
check = False
    #export jpg
''''

In [ ]:
'''
#right_cluster,left_cluster = cluster()

#pose = 1
right_cluster,left_cluster = cluster()
cap = cv.VideoCapture(0)
cap.set(cv.CAP_PROP_AUTOFOCUS, 0)

ret, Background = cap.read()
flag = False
while(True):
    ret, frame = cap.read()

    bin_img = segmentation(frame,Background)
    
    #border extraction step from bin_img
    #border = border_extraction(bin_img)
    borderImg,check = border_extraction(bin_img)

    
    if(check == True):
        
        xc,yc,y_head,leg_height,flag2 = centroid_2(borderImg)
        

        #star skeletonization step
        if(flag2):
            left_arm,head,right_arm,flag2= star(borderImg,xc,yc,y_head)
            
        #get angles
        #distinguish points 
        #right_arm,left_arm,head = distinguish_points(star_points,centroid)
        #print(flag2)
        #show points
        if(flag2):
            col_img =gray2rgb(borderImg)
            #print(((int)(right_arm[0]),(int)(right_arm[1])))
            #print(((int)(left_arm[0]),(int)(left_arm[1])))
            cv.circle(frame,((int)(right_arm[0]),(int)(right_arm[1])),10,(255,0,255),2)
            cv.circle(frame,((int)(left_arm[0]),(int)(left_arm[1])),10,(255,0,255),2)
            cv.circle(frame,((int)(head[0]),(int)(head[1])),10,(255,0,255),2)
            window = cv.namedWindow("frame", cv.WINDOW_NORMAL| cv.WINDOW_KEEPRATIO )
            cv.imshow("frame", frame)
    check = False
    #export jpg
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()
'''

In [ ]:
'''
def cluster():
    image_list = []
    right_list = []
    left_list = []
    name_list = []
    
    background = io.imread ("T.1_back.jpg")
    
    for filename in glob.glob('pictures/*.jpg'): #assuming gif
        im=io.imread(filename)
        image_list.append(im)
        io.imshow(im)
        io.show()
        right_point,left_point,out_img = cluster_sequence(im,background)
        right_list.append(right_point)
        
        name_list.append(filename)
        left_list.append(left_point)
        io.imsave('star_'+filename,out_img)

    background = io.imread ("blck.jpg")
    for filename in glob.glob('human/*.jpg'): #assuming gif
        im=io.imread(filename)
        image_list.append(im)
        
        right_point,left_point,out_img = cluster_sequence(im,background)
        right_list.append(right_point)
      
        name_list.append(filename)
        left_list.append(left_point)
        io.imsave('star_'+filename,out_img)
#
    for filename in glob.glob('salma/*.jpg'): #assuming gif
        im=io.imread(filename)
        image_list.append(im)
        
        right_point,left_point,out_img = cluster_sequence(im,background)
        right_list.append(right_point)
    
        name_list.append(filename)
        left_list.append(left_point)
        io.imsave('star_'+filename,out_img)

    cluster_n = 5
    
    right_cluster=KMeans(cluster_n,random_state=0).fit(right_list)
    
    left_cluster=KMeans(cluster_n,random_state=0).fit(left_list)
    predictError(name_list,right_list,right_cluster,left_list,left_cluster)
    
    return right_cluster,left_cluster
    '''